In [5]:
import requests
import json
import bs4
import re
import pandas as pd
from tqdm import tqdm

def AssetAllocationInfo(SecId, headers):
  url = f"https://www.us-api.morningstar.com/sal/sal-service/fund/process/asset/v2/{SecId}/data?languageId=zh-HK&locale=zh-HK&clientId=MDC_intl&benchmarkId=mstarorcat&component=sal-components-mip-asset-allocation&version=3.60.0"
  res = requests.get(url, headers = headers)
  data = json.loads(res.content.decode("utf-8"))
  allocation = []
  percent = []
  for i in data['allocationMap']:
    allocation.append(i)
    percent.append(data['allocationMap'][i]["netAllocation"])
  return allocation, percent

def get_token(SecId):
  url = f"https://www.morningstar.hk/hk/report/fund/portfolio.aspx?t={SecId}&fundservcode=&lang=zh-HK"
  res = requests.get(url)
  token = re.findall("maas_token=.*content", str(bs4.BeautifulSoup(res.content.decode("utf-8")).find(name="sal-components")))[0][12:-13]
  return token

In [6]:
df = pd.read_csv("YOUR_FILENAME_HERE.csv")   

In [16]:
def get_extra_info(df):
  for SecId in df.SecId.to_list()[:1]:
    token = get_token(SecId)
    token = f"Bearer {token}"
    headers = {"authorization": token}
    print(AssetAllocationInfo(SecId, headers))
  
get_extra_info(df)

(['AssetAllocCash', 'AssetAllocNotClassified', 'AssetAllocNonUSEquity', 'AssetAllocOther', 'AssetAllocUSEquity', 'AssetAllocBond'], ['-119.68299', '15.095040000000001', '34.35388', '0.66375', '61.71301', '107.85719'])
